In [1]:
!pip install faiss-cpu

In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [3]:
from retrievers.DPR import DPRRetriever
from Evaluation import Evaluation

In [4]:
doc_path = "../dataset/CISI.ALL"
qry_path = "../dataset/CISI.QRY"
rel_path = "../dataset/CISI.REL"

dpr_retriever = DPRRetriever(doc_path, qry_path, rel_path)

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

In [5]:
retrieved_docs = {}
relevant_docs = dpr_retriever.rel_set

for _, row in dpr_retriever.qry_set.iterrows():
    qid = row["query_id"]
    qtext = row["text"]

    num_relevant_docs = len(relevant_docs.get(qid, set()))
    retrieved_results = dpr_retriever._retrieve_dpr(qtext)
    sliced_retrieved_results = retrieved_results[:num_relevant_docs]
    retrieved_docs[qid] = [result["doc_id"] for result in sliced_retrieved_results]

retrieved_docs = {str(qid): [str(doc_id) for doc_id in docs] for qid, docs in retrieved_docs.items()}
relevant_docs = {str(qid): {str(doc_id) for doc_id in docs} for qid, docs in relevant_docs.items()}

In [6]:
retrieved_docs = {str(qid): [str(doc_id) for doc_id in docs] for qid, docs in retrieved_docs.items()}
relevant_docs = {str(qid): {str(doc_id) for doc_id in docs} for qid, docs in relevant_docs.items()}

print(retrieved_docs)

query_ids = list(relevant_docs.keys())
evaluator = Evaluation(retrieved_docs, relevant_docs, query_ids)

results_file = '../results/dpr_evaluation.txt'
evaluator.save_evaluation_results(results_file)

{'1': ['851', '258', '420', '869', '898', '1210', '1144', '457', '619', '564', '28', '724', '746', '1091', '1280', '966', '333', '582', '578', '474', '784', '799', '589', '1304', '1259', '826', '315', '661', '1266', '867', '722', '88', '257', '419', '576', '805', '178', '643', '29', '1013', '1446', '282', '657', '503', '1413', '381'], '2': ['318', '851', '869', '876', '1280', '1327', '457', '1013', '420', '799', '1078', '683', '1366', '724', '867', '966', '258', '633', '576', '142', '743', '680', '1089', '855', '578', '419'], '3': ['318', '258', '1280', '1089', '463', '513', '851', '494', '472', '1102', '1013', '345', '1327', '137', '1078', '1297', '966', '139', '60', '1289', '456', '474', '526', '179', '1282', '535', '867', '175', '1413', '960', '898', '1284', '79', '856', '743', '419', '381', '574', '871', '576', '488', '363', '142', '658'], '4': ['1280', '867', '1327', '420', '980', '851', '503', '1193'], '5': ['1327', '1013', '619', '851', '29', '1280', '178', '1293', '548', '1078'